In [2]:
import pandas as pd
import time
import re
import datetime

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urljoin
from konlpy.tag import Okt, Hannanum
from tqdm.notebook import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [28]:
def search(name_list):
    driver = webdriver.Chrome('chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://search.naver.com/search.naver?where=news')
    
    driver.find_element_by_name('query').send_keys(name_list)
    driver.find_element_by_class_name('bt_search').click()
    
    title = []
    news_date = []
    word_data = []
    
    date_re = re.compile(r'\d\d\d\d.\d\d.\d\d.')
    
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        title_list = soup.find_all(class_='_sp_each_title')
        date = soup.find_all(class_='txt_inline')
        word = soup.find_all(class_='type01')[0].find_all('dl')

        for i in title_list:
            title.append(i.get_text())

        for dates in date:
            try:
                match = date_re.search(dates.get_text())
                news_date.append(match.group())
            except:
                date_re2 = re.compile(r'\d')
                match = date_re2.search(dates.get_text())
                m = int(match.group())
                year = str(datetime.datetime.today().year)
                month = str(datetime.datetime.today().month)
                day = str(datetime.datetime.today().day-m)
                result = year + '.' + month + '.' + day + '.'
                news_date.append(result)

        for words in word:
            word_data.append(words.find_all('dd')[1].get_text())
            
        try:
            if soup.find_all(class_='next')[0].get_text() != '다음페이지':
                break
            else: 
                driver.find_element_by_class_name('next').click()
                driver.implicitly_wait(1)
        except:
            if soup.find(class_='next') != '다음페이지':
                break
    
    df = pd.DataFrame(list(zip(title, news_date, word_data)), columns = ['title', 'date','contents'])
    driver.close()
    
    return df

In [5]:
def sentence(df):
    
    df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")
    
    stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    hannanum = Hannanum()
    
    contents = []

    for sentence in tqdm(df['contents']):
        temp_x = []
        temp_x = hannanum.nouns(sentence)
        temp_x = [word for word in temp_x if not word in stopwords]
        contents.append(temp_x)
        
    return contents


In [6]:
def tokenizers(contents):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(contents)
    sequences = tokenizer.texts_to_sequences(contents)
    
    word_to_index = tokenizer.word_index
    index_to_word = {}
    for key, value in word_to_index.items():
        index_to_word[value] = key
        
    return index_to_word


In [7]:
def word_count(contents):
    wordlist = sum(contents, [])
    word_counts = {}
    
    for word in wordlist:
        word_counts[word] = word_counts.get(word, 0) + 1
        keys = sorted(word_counts.keys())
        
    for word in keys:
        print(word + ':' + str(word_counts[word]))

In [29]:
name_list1 = '서구의료재단여수성심병원'
name_list2 = '고신대복음병원'

In [30]:
bankrup1 = search(name_list1)

In [10]:
bankrup1

,title,date,contents
0,여수성심병원 156억에 경매…병원기능 소멸 위기,2020.04.22.,22일 여수시에 따르면 의료법인 서구의료재단의 여수성심병원 건물 10개 동과 부지 ...
1,"""건설사에 넘어간 여수성심병원···'의료시설부지→주택단지'로 개발?""",2020.04.09.,"의료법인 서구의료재단이 운영하는 여수성심병원은, 극심한 경영난을 이겨내지 못하고 지..."
2,여수 성심병원 경영 악화로 오늘부터 휴업,2018.07.23.,[전남CBS 성심병원 홈페이지의 병원 휴업 공지(사진=성심병원 제공)여수 성심병원(...
3,여수성심병원 향후 활용방안 관심,2020.06.02.,2일 지역 의료계에 따르면 지난 4월 서구의료재단 여수 성심병원 건물 10개 동과 ...
4,'휴업 2년' 여수 성심병원 주인 바뀌나,2020.04.16.,16일 지역 의료계에 따르면 지난 6일 광주지법 순천지원 경매법정에서 의료법인 서구...
...,...,...,...
131,"[18일 낙찰/계약 동향] 남산4동 까치아파트 서편 도로 및 하수도 정비공사, 경...",2019.09.18.,"계약기관 : 재단법인 한국도자재단, 계약일자 : 2019-09-11, 계약방법 : ..."
132,"[29일 조달청 입찰정보/종합] 2020년 학생 통계교육 과정 운영 위탁, 경복대학...",2020.01.29.,"▲ 입찰명 : ""세종충남대병원 [의료기기] 산소포화도측정기(일반성인용)1SET외 1..."
133,"[9일 낙찰/계약 동향] 예당권역종합정비사업 목욕장설비 설치공사, 춘천시 청...",2019.09.09.,"경찰병원 65병동 시설개선공사(건축,기계)). ▲ 낙찰업체 : 성심이앤지, 계약금액..."
134,"[15일 낙찰/계약 동향] 안암초 방송실 환경개선공사(인테리어 공사), 악양생태...",2019.08.15.,"▲ 낙찰업체 : 주식회사 성심산업, 계약금액 : 19,620,000(원), 계약명 ..."


In [31]:
bankrup1_contents = sentence(bankrup1)

In [32]:
wordlist = sum(bankrup1_contents, [])
word_list = pd.Series(wordlist)
result = word_list.value_counts()

In [33]:
result.to_csv('부도기업1.csv', mode='w')

In [34]:
# bankrup1_contents = tokenizers(bankrup1_contents)

In [35]:
# with open('bankrup1.txt', 'w') as f:
#     for b1c in bankrup1_contents.values():
#        f.writelines(b1c+'\n')

In [36]:
bankrup2  = search(name_list2)

In [37]:
bankrup2_contents = sentence(bankrup2)

In [38]:
bankrup2_contents

[['부산일보',
  '고신대복음병원',
  '외',
  '전창완',
  '교수',
  '유방',
  '전체',
  '절제술',
  '대신',
  '부분절제술',
  '시행',
  '비율',
  '유방',
  '보존율',
  '선행',
  '유도',
  '항암요법',
  '적극적',
  '시도',
  '부산'],
 ['식품',
  '알레르기',
  '세',
  '선진국형',
  '질환',
  '삶',
  '질',
  '서구',
  '선진국',
  '관심',
  '분야',
  '우리나라',
  '몇',
  '년',
  '연구',
  '고신대복음병원',
  '년',
  '부산광역시'],
 ['고신대복음병원',
  '병원장',
  '최영식',
  '소아청소년',
  '정민영',
  '교수',
  '연구과제',
  '일',
  '과학기술정보통신부',
  '주관한',
  '비대',
  '비즈니스',
  '디지털혁신기술개발사업',
  '의료분야',
  '공모사업',
  '선정',
  '학교',
  '측이',
  '일'],
 ['고신대복음병원',
  '년',
  '부산광역시',
  '아토피천식교육정보센터',
  '년',
  '운영',
  '아토피',
  '알레르기',
  '질환',
  '진료데이터',
  '연구역량',
  '노하우',
  '지속적',
  '식품제한',
  '필요',
  '아토피질환'],
 ['고신대복음병원',
  '병원장',
  '최영식',
  '한국마이크로스프트',
  '실시간',
  '커뮤니케이션',
  '서비스',
  '팀즈',
  '도입',
  '업무',
  '협약식',
  '지난달',
  '일',
  '장기려기념암센터',
  '대회의실',
  '고신대복음병원',
  '원격근무'],
 ['고신대복음병원',
  '췌담도내',
  '박은택',
  '교수팀',
  '특수',
  '내시경',
  '췌관',
  '담관',
  '병변',
  '촬영',
  '치료',
  '시술',
  '부산일보',
  '박은택',
  '고신대복음병원',
  '췌담도내',
  '

In [39]:
wordlist2 = sum(bankrup2_contents, [])
word_list2 = pd.Series(wordlist2)
result = word_list2.value_counts()
result

고신대복음병원    4555
일          2967
병원장        1939
교수         1021
최영식         917
           ... 
소상공인          1
개선공사          1
공공기관          1
블루크로스         1
헬리코박터균        1
Length: 7539, dtype: int64

In [40]:
result.to_csv('건실기업1.csv', mode='w')

In [41]:
# with open('bankrup2.txt', 'w') as f:
#     for b2c in bankrup2_contents.values():
#        f.writelines(b2c+'\n')

In [42]:
name_list3 = '애민의료재단'
name_list4 = '성석의료재단'

In [43]:
bankrup3 = search(name_list3)

In [44]:
bankrup4 = search(name_list4)

In [45]:
bankrup3_contents = sentence(bankrup3)
bankrup4_contents = sentence(bankrup4)

In [46]:
wordlist3 = sum(bankrup3_contents, [])
word_list3 = pd.Series(wordlist3)
result = word_list3.value_counts()
result.to_csv('부도기업2.csv', mode='w')

In [47]:
wordlist4 = sum(bankrup4_contents, [])
word_list4 = pd.Series(wordlist4)
result = word_list4.value_counts()
result.to_csv('부도기업3.csv', mode='w')